# Nursing Home Data - Scraping from PDF
Nursing homes in the Dudley area has been provided by [Dudley council](https://www.dudley.gov.uk/residents/care-and-health/adult-health-social-care/housing-with-care-and-support/care-homes-residential-and-nursing/) in a PDF avalaible in this [link](https://www.dudley.gov.uk/media/ktclgusy/2023-24_approved_care_home_providers_within_the_dudley_borough_available_to_the_public.pdf) 

In [66]:
# Import of the necessary libraries
import pandas as pd
import numpy as np
import camelot
import re

# Temporarily set max rows to 100
pd.set_option('display.max_rows', 100)

# Set the 1st column width 100
pd.set_option('display.max_colwidth', 100)


# Using the Camelot Libary to extarct the data from the PDF

In [67]:
# Read all tables from all pages
tables = camelot.read_pdf('../data/nhomes.pdf', pages='all')

# Initialize an empty DataFrame
all_tables = pd.DataFrame()

# Loop through each table and append it to the all_tables DataFrame
for table in tables:
    df = table.df  # get the table as a DataFrame
    df = df.iloc[1:]  # skip the first row
    all_tables = pd.concat([all_tables, df])

# Reset the index of the final DataFrame
all_tables.reset_index(drop=True, inplace=True)

# Label the columns
headers = tables[1].df.iloc[0]
all_tables.columns = headers

In [89]:
# Create a copy of the dataframe
all_tables_copy = all_tables.copy()

# Function to make the header row all into camelCase
def to_camel_case(s):
    s = s.title().replace(' ', '')
    return s[0].lower() + s[1:]

# Apply the function to the headers and remove '\n'
all_tables_copy.columns = all_tables_copy.columns.str.replace('\n', '')
all_tables_copy.columns = all_tables_copy.columns.map(to_camel_case)


## Data Cleaning

Name and Address Column

In [90]:
# Split the 'Name & Address' column into 'Post Code' and 'Care Home Name'
all_tables_copy['postCode'] = all_tables_copy['name&Address'].str.split('\n').str[-1]
all_tables_copy['careHomeName'] = all_tables_copy['name&Address'].str.split('\n').str[0]

# Replace the '\n' from the Name a& Address column
all_tables_copy['name&Address'] = all_tables_copy['name&Address'].str.replace('\n', ',')
# all_tables_copy['Name & Address'] = all_tables_copy['Name & Address'].str.replace('  ', ' ')

# Remove the space before the comma in the 'Name & Address' column
all_tables_copy['name&Address'] = all_tables_copy['name&Address'].str.replace(' ,', ',')

Email Column

In [91]:
# Repalce the '\n' form the 'email' column
all_tables_copy['email'] = all_tables_copy['email'].str.replace('\n', '')

# Extract the telephone number from the 'email' column
all_tables_copy['telephoneNo'] = np.where(all_tables_copy['email'].str.contains(' (\d+.*$)'),
                             all_tables_copy['telephoneNo'] + re' (\d+.*$)',
                             all_tables_copy['telephoneNo'])
all_tables_copy['email'] = all_tables_copy['email'].str.replace(r'\s+\d+$', '', regex=True)


/var/folders/05/1f_ky4t905q0hj0yjgf_xkd80000gn/T/ipykernel_4038/690241558.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_tables_copy['telephoneNo'] = np.where(all_tables_copy['email'].str.contains(' (\d+.*$)'),


Telephone Coloumn

In [92]:
#Remove non-numeric characters from the 'telephoneNo' column
all_tables_copy['telephoneNo'] = all_tables_copy['telephoneNo'].str.replace(r'\D', '', regex=True)

Age Column

In [64]:
# Splitting the Age Range column into 'Min Age' and 'Max Age'
all_tables_copy['minAge'] = all_tables_copy['agerange'].str.extract('(\d{2})')
all_tables_copy['maxAge'] = all_tables_copy['agerange'].str.extract('-(\d{2})')

CQC Registered For Column

In [96]:
# Cleaning the 'Age Range' column
# Move the 'Dementia' word to the 'CQC Registered For' column
all_tables_copy['cqcRegisteredFor'] = np.where(all_tables_copy['ageRange'].str.contains('Dementia'),
                             all_tables_copy['cqcRegisteredFor'] + ', Dementia',
                             all_tables_copy['cqcRegisteredFor'])
all_tables_copy['ageRange'] = all_tables_copy['ageRange'].str.replace('Dementia', '')

# Move the 'Old Age' word to the 'cqcRegisteredFor' column
all_tables_copy['cqcRegisteredFor'] = np.where(all_tables_copy['ageRange'].str.contains('Old Age'),
                             all_tables_copy['cqcRegisteredFor'] + ', Old Age',
                             all_tables_copy['cqcRegisteredFor'])
all_tables_copy['ageRange'] = all_tables_copy['ageRange'].str.replace('Old Age', '')

# Replace all the '\n' from the 'CQC Registered For' column with ','
all_tables_copy['cqcRegisteredFor'] = all_tables_copy['cqcRegisteredFor'].str.replace('\n', ',')

In [100]:
# Save the final DataFrame to a CSV file
all_tables_copy.to_csv('../data/nhomes_cleaned.csv', index=False)

In [99]:
all_tables_copy.tail(100)

,name&Address,email,telephoneNo,ageRange,cqcRegisteredFor,provideNursing,capacity,postCode,careHomeName
0,"Abbeygate Care,Centre,2 Leys Road,Brockmoor,Brierley Hill,DY5 3UR",abbeygatecare1@gmail.com,01384571295,Age 65+,"Dementia ,Mental Health Condition ,Old Age ,Physical Disability ,Sensory Impairment",No,17,DY5 3UR,Abbeygate Care
1,"Abbeymere,12 Eggington Road,Wollaston,Stourbridge,DY8 2QJ",abbeymere@karelink.co.uk,01384395195,Ages 65+,"Dementia ,Mental Health Condition ,Old Age ,Physical Disability ,Sensory Impairment",No,18,DY8 2QJ,Abbeymere
2,"Allenbrook Nursing,Home,209 Spies Lane,Halesowen,B62 9SJ",manager@allenbrooknursinghome.co.uk,01214225844,Ages 55+,"Dementia ,Mental Health Condition ,No Medical Intervention ,Old Age ,Physical Disability ,Sensor...",Yes,36,B62 9SJ,Allenbrook Nursing
3,"Amberley Care Home,481-483 Stourbridge,Road,Harts Hill,Brierley Hill,DY5 1LB",amberleycarehome@hotmail.co.uk,01384482365,Ages 65+,"Dementia ,Old Age",No,25,DY5 1LB,Amberley Care Home
4,"Arcare For Forte,440 Birmingham New,Road,Bilston,WV14 9QB",ksharma@arcarehomes.co.uk,01902880108,18+,"Learning Disability ,Mental Health Condition ,Old Age ,Physical Disability ,Sensory Impairment ,...",No,9,WV14 9QB,Arcare For Forte
5,"Ashbourne Care Ltd,Lightwood Road,Dudley,DY1 2RS",ashbourne.m@fshc.co.uk,01384242200,Ages 65+,"Dementia ,Old Age",No,38,DY1 2RS,Ashbourne Care Ltd
6,"Ashgrove Nursing,Home,9 Dudley Wood Road,Netherton,Dudley,DY2 0DA",cea@ashgrovecare.com,01384413913,Ages 65+,"Dementia ,Old Age ,Sensory Impairment",Yes,57,DY2 0DA,Ashgrove Nursing
7,"Avondale ,45 Norton Road,Norton,Stourbridge,DY8 2AH",avondaleresthome@hotmail.com,01384442731,Ages 65+,Old Age,No,15,DY8 2AH,Avondale
8,"Beatrice House,25 Bell Street,Pensnett,Brierley Hill,DY5 4HG",beatricehouse@alphonsusservices.co.uk,01384482963,Not Stated,Learning Disability,No,3,DY5 4HG,Beatrice House
9,"Belvidere,41-43 Stourbridge,Road,Holly Hall,Dudley,DY1 2DH",belvidere@gmail.com,01384211850,Ages 55+,"Dementia ,Detention Under Mental ,Health Act ,Mental Health Condition ,Old Age ,Physical Disabil...",No,28,DY1 2DH,Belvidere
